In [1]:
import foam_utilities as foam 
import os, shutil
import subprocess
import multiprocessing

import numpy as np
import pandas as pd
import random
np.random.seed(0)

In [2]:
# geometry parameter
aw=1

In [3]:
# number of datapoints used as input in a stencil
stencil_size = 60

In [4]:
# coefficient in equation
epsilon = 0.2
diff = 0.02
zeta = 3

In [5]:
# number of cells in x and y direction
ncellx = round(aw * 200)
ncelly = int(200)
ncell = int(ncellx * ncelly)
# geometry
scale = 0.0357143
Lx = (252 - 54 * (1-aw) * 2) * scale

In [6]:
# directory location
caseBase = 'transport_'
caseDir = caseBase + str(aw)
timeDir1 = '10'
timeDir2 = '0'

In [7]:
# read data from directory

ufile = os.path.join(caseDir, timeDir1, 'U') # velocity
U = foam.read_vector_field(ufile)

cfile = os.path.join(caseDir, timeDir1, 'C') # cell centers
Cc = foam.read_vector_field(cfile)

strainfile = os.path.join(caseDir, timeDir1, 'S')  # strain rate magnitude
strain = foam.read_scalar_field(strainfile)

cellVolumefile = os.path.join(caseDir, timeDir1, 'V')  # cellVolume
cellVolume = foam.read_scalar_field(cellVolumefile)

wDfile = os.path.join(caseDir, timeDir2, 'wallDistance') # wall distance 
wD = foam.read_scalar_field(wDfile)

boundary = np.zeros([ncell,1]) # boundary information (Yes: 1; No: 0)
boundary[0:ncellx,:] = np.ones([ncellx,1])
boundary[(ncell-ncellx):ncell,:] = np.ones([ncellx,1])

tfile = os.path.join(caseDir, timeDir1, 'T') # temperature
T = foam.read_scalar_field(tfile)

In [8]:
# generate input data
data = np.zeros([ncell,9])

data[:,0:2] = Cc[:,0:2]
data[:,2:4] = U[:,0:2]
data[:,4:5] = strain.reshape(ncell,1)
data[:,5:6] = wD.reshape(ncell,1)
data[:,6:7] = boundary
data[:,7:8] = T.reshape(ncell,1)
data[:,8:9] = cellVolume.reshape(ncell,1)
df = pd.DataFrame(data,columns=['Cx','Cy','u','v','strain','wD','boundary','T','cellVolume'])

In [9]:
df

,Cx,Cy,u,v,strain,wD,boundary,T,cellVolume
0,0.024021,1.002170,0.011572,6.128200e-05,5.68513,0.002043,1.0,0.028127,0.000011
1,0.072063,1.002310,0.011627,3.791000e-06,5.68044,0.002043,1.0,0.027621,0.000011
2,0.120097,1.001660,0.011593,-3.175050e-04,5.64296,0.002043,1.0,0.026996,0.000011
3,0.168056,0.999126,0.011193,-8.687120e-04,5.45668,0.002039,1.0,0.026031,0.000010
4,0.215749,0.993652,0.010051,-1.508400e-03,4.97062,0.002027,1.0,0.024614,0.000010
...,...,...,...,...,...,...,...,...,...
39995,8.797480,3.033660,0.006559,3.945510e-07,3.18216,0.002052,1.0,0.012154,0.000010
39996,8.842490,3.033670,0.006584,3.756950e-07,3.20258,0.002046,1.0,0.012246,0.000010
39997,8.887490,3.033670,0.006616,3.919450e-07,3.22260,0.002044,1.0,0.012352,0.000010
39998,8.932500,3.033670,0.006652,3.468120e-07,3.24111,0.002043,1.0,0.012466,0.000010


In [10]:
df['U']=list(map(lambda x,y: np.sqrt(x**2+y**2), df['u'], df['v'])) # add column of velocity mag "U"
Umax = df.loc[:,"U"].max()
outlength = np.abs(2*diff*np.log(epsilon)/(Umax-np.sqrt(Umax**2+4*diff*zeta))) # region outside the domain because of periodic hills
dfout1 = df[(df['Cx'] > 0) & (df['Cx'] < outlength)]
dfout2 = df[(df['Cx'] > (Lx - outlength)) & (df['Cx'] < Lx)]
dfout1new = dfout1.copy(deep=True)
dfout1new['Cx'] = dfout1new['Cx'] + Lx
dfout2new = dfout2.copy(deep=True)
dfout2new['Cx'] = dfout2new['Cx'] - Lx
dfdata = df.append(dfout1new,ignore_index=True)
dfdata = dfdata.append(dfout2new,ignore_index=True)

In [11]:
outlength

0.7534930522550446

In [12]:
Ly = np.sqrt(diff/zeta)*np.abs(np.log(epsilon)) # distance of diffusion only

In [13]:
Ly

0.13141005527178998

In [14]:
# define wall distance function
def wDfunc(wallDistance):
    if wallDistance > 1.5*Ly:
        wallfunc = 1.0
    else:
        wallfunc = wallDistance/(1.5*Ly)
        
    return wallfunc

In [15]:
dfdata['wallinfo']=list(map(lambda x: wDfunc(x), dfdata['wD'])) # add column of wall information (function of wD)
dfdata = dfdata.drop(columns=['wD']) # delete the original wall distance column

In [16]:
# define the nonlocal region from local information (u,v,Cx,Cy)
def ellipse(x,y,u,v,a,b,Cx,Cy): 
    alpha = np.arctan(v/u)
    xp = (x-Cx)*np.cos(alpha)+(y-Cy)*np.sin(alpha)
    yp = (x-Cx)*np.sin(alpha)-(y-Cy)*np.cos(alpha)
    value = xp**2/a**2+yp**2/b**2
    return value

In [17]:
# define the influence of the relative position and velocity of certain point to the center point
def rfinalfunc(Cx,Cy,u,v):
    rmag = np.sqrt(Cx**2+Cy**2)
    rfun = 0.01/(rmag+0.01)
    umag = np.sqrt(u**2+v**2)
    cos = -1*(Cx*u+Cy*v)/(rmag*umag+1e-10)
    rfinal = 0.5*(cos+1.05)*umag*rfun
    return rfinal

In [18]:
# get nonlocal datapoints
data_ind=0
dataX_current=np.zeros([ncellx*ncelly, stencil_size, 11])
dataY_current=np.zeros([ncellx*ncelly, 1])

In [20]:
for row in dfdata.itertuples():
    # define the local elliptical region
    ellx = np.abs(2*diff*np.log(epsilon)/(getattr(row,'U')-np.sqrt(getattr(row,'U')**2+4*diff*zeta)))
    elly = np.sqrt(diff/zeta)*np.abs(np.log(epsilon))
    
    # take points in the region
    df1 = dfdata[ellipse(dfdata['Cx'],dfdata['Cy'],getattr(row,'u'),getattr(row,'v'),ellx,elly,getattr(row,'Cx'),getattr(row,'Cy')) < 1.0]
    
    # calculate the relative coordinate
    df2 = df1.copy(deep=True)
    df2['Cx'] = df2['Cx'] - getattr(row,'Cx')
    df2['Cy'] = df2['Cy'] - getattr(row,'Cy')
    
    # add a new column of relative distance to the centre point
    df3 = df2.copy(deep=True)
    df3['rD']=list(map(lambda x,y: np.sqrt(x**2+y**2), df3['Cx'], df3['Cy']))
    
    # rearrange the points (relative distance: from small to large)
    df4 = df3.copy(deep=True)
    df4.sort_values("rD",inplace=True)
    
    # uniform sampling from the points in the local elliptical region
    if len(df4) == stencil_size:
        df5 = df4
    elif len(df4) > stencil_size:
        df5 = df4.sample(n=stencil_size,random_state=1,replace=False)
    else:
        df4out = df4.sample(n=(stencil_size-len(df4)),random_state=1,replace=True)
        df5 = df4.append(df4out,ignore_index=True)
    
    # normalize the cell volume
    df6 = df5.copy(deep=True)
    V_max = df6.loc[:,"cellVolume"].max()
    df6['cellVolume'] = df6['cellVolume'] / V_max
    
    df7 = df6.drop(columns=['T']) # drop the label column
    df7['rfianl']=list(map(lambda x,y,u,v: rfinalfunc(x,y,u,v),df7['Cx'],df7['Cy'],df7['u'],df7['v'])) 
    
    dataX_current[data_ind,:,:]=df7.to_numpy()
    dataY_current[data_ind,0]=getattr(row,'T')
    data_ind+=1
        
    if getattr(row,'Index')==(ncellx*ncelly-1):  
        break

In [21]:
df1

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo
98,4.42794,0.003047,-0.002169,-4.622950e-07,0.701232,1.0,0.001359,0.000016,0.002169,0.015459
99,4.47598,0.003047,-0.002137,-5.023570e-07,0.690911,1.0,0.001349,0.000016,0.002137,0.015459
100,4.52402,0.003047,-0.002105,-5.414760e-07,0.680332,1.0,0.001340,0.000016,0.002105,0.015459
101,4.57207,0.003047,-0.002072,-5.796390e-07,0.669509,1.0,0.001330,0.000016,0.002072,0.015459
102,4.62011,0.003047,-0.002038,-6.167920e-07,0.658459,1.0,0.001321,0.000016,0.002038,0.015459
...,...,...,...,...,...,...,...,...,...,...
3301,4.57190,0.114418,-0.055039,-1.389850e-03,0.284241,0.0,0.044451,0.000020,0.055057,0.580468
3499,4.47604,0.122388,-0.059501,-1.522860e-03,0.273740,0.0,0.047315,0.000021,0.059521,0.620896
3500,4.52396,0.122388,-0.058340,-1.553450e-03,0.265592,0.0,0.047279,0.000021,0.058361,0.620896
3501,4.57189,0.122388,-0.057160,-1.582000e-03,0.257381,0.0,0.047246,0.000021,0.057182,0.620896


In [22]:
df2

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo
98,-0.09608,0.000000,-0.002169,-4.622950e-07,0.701232,1.0,0.001359,0.000016,0.002169,0.015459
99,-0.04804,0.000000,-0.002137,-5.023570e-07,0.690911,1.0,0.001349,0.000016,0.002137,0.015459
100,0.00000,0.000000,-0.002105,-5.414760e-07,0.680332,1.0,0.001340,0.000016,0.002105,0.015459
101,0.04805,0.000000,-0.002072,-5.796390e-07,0.669509,1.0,0.001330,0.000016,0.002072,0.015459
102,0.09609,0.000000,-0.002038,-6.167920e-07,0.658459,1.0,0.001321,0.000016,0.002038,0.015459
...,...,...,...,...,...,...,...,...,...,...
3301,0.04788,0.111371,-0.055039,-1.389850e-03,0.284241,0.0,0.044451,0.000020,0.055057,0.580468
3499,-0.04798,0.119341,-0.059501,-1.522860e-03,0.273740,0.0,0.047315,0.000021,0.059521,0.620896
3500,-0.00006,0.119341,-0.058340,-1.553450e-03,0.265592,0.0,0.047279,0.000021,0.058361,0.620896
3501,0.04787,0.119341,-0.057160,-1.582000e-03,0.257381,0.0,0.047246,0.000021,0.057182,0.620896


In [23]:
df3

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo,rD
98,-0.09608,0.000000,-0.002169,-4.622950e-07,0.701232,1.0,0.001359,0.000016,0.002169,0.015459,0.096080
99,-0.04804,0.000000,-0.002137,-5.023570e-07,0.690911,1.0,0.001349,0.000016,0.002137,0.015459,0.048040
100,0.00000,0.000000,-0.002105,-5.414760e-07,0.680332,1.0,0.001340,0.000016,0.002105,0.015459,0.000000
101,0.04805,0.000000,-0.002072,-5.796390e-07,0.669509,1.0,0.001330,0.000016,0.002072,0.015459,0.048050
102,0.09609,0.000000,-0.002038,-6.167920e-07,0.658459,1.0,0.001321,0.000016,0.002038,0.015459,0.096090
...,...,...,...,...,...,...,...,...,...,...,...
3301,0.04788,0.111371,-0.055039,-1.389850e-03,0.284241,0.0,0.044451,0.000020,0.055057,0.580468,0.121227
3499,-0.04798,0.119341,-0.059501,-1.522860e-03,0.273740,0.0,0.047315,0.000021,0.059521,0.620896,0.128625
3500,-0.00006,0.119341,-0.058340,-1.553450e-03,0.265592,0.0,0.047279,0.000021,0.058361,0.620896,0.119341
3501,0.04787,0.119341,-0.057160,-1.582000e-03,0.257381,0.0,0.047246,0.000021,0.057182,0.620896,0.128584


In [24]:
df4

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo,rD
100,0.00000,0.000000,-0.002105,-5.414760e-07,0.680332,1.0,0.001340,0.000016,0.002105,0.015459,0.000000
300,0.00000,0.006144,-0.006221,-8.613790e-06,0.659199,0.0,0.004034,0.000016,0.006221,0.046631,0.006144
500,0.00000,0.012390,-0.010271,-2.538600e-05,0.637627,0.0,0.006755,0.000016,0.010271,0.078314,0.012390
700,-0.00001,0.018737,-0.014249,-5.137170e-05,0.615609,0.0,0.009491,0.000016,0.014249,0.110516,0.018737
900,-0.00001,0.025189,-0.018148,-8.668790e-05,0.593149,0.0,0.012233,0.000017,0.018148,0.143245,0.025189
...,...,...,...,...,...,...,...,...,...,...,...
3700,-0.00006,0.127442,-0.060336,-1.757060e-03,0.238407,0.0,0.050118,0.000021,0.060361,0.661994,0.127442
3501,0.04787,0.119341,-0.057160,-1.582000e-03,0.257381,0.0,0.047246,0.000021,0.057182,0.620896,0.128584
3499,-0.04798,0.119341,-0.059501,-1.522860e-03,0.273740,0.0,0.047315,0.000021,0.059521,0.620896,0.128625
2702,0.09587,0.088225,-0.046695,-9.135640e-04,0.354455,0.0,0.036227,0.000019,0.046704,0.463040,0.130287


In [27]:
V_max

2.09609e-05

In [25]:
df5

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo,rD
301,0.04804,0.006144,-0.006122,-8.807180e-06,0.648501,0.0,0.004006,0.000016,0.006122,0.046631,0.048431
2298,-0.09597,0.073409,-0.044150,-5.957790e-04,0.442379,0.0,0.031424,0.000019,0.044154,0.387876,0.120827
698,-0.09605,0.018737,-0.014698,-4.892190e-05,0.635802,0.0,0.009620,0.000016,0.014698,0.110516,0.097861
3100,-0.00005,0.103530,-0.053781,-1.189460e-03,0.319533,0.0,0.041797,0.000020,0.053794,0.540684,0.103530
2900,-0.00005,0.095815,-0.051236,-1.028280e-03,0.346163,0.0,0.039117,0.000020,0.051247,0.501546,0.095815
1898,-0.09599,0.059067,-0.037456,-3.959980e-04,0.493188,0.0,0.026090,0.000018,0.037458,0.315119,0.112708
2901,0.04790,0.095815,-0.050260,-1.048050e-03,0.337362,0.0,0.039014,0.000020,0.050271,0.501547,0.107121
2698,-0.09595,0.088225,-0.050293,-8.426110e-04,0.390095,0.0,0.036697,0.000019,0.050300,0.463039,0.130346
1902,0.09594,0.059067,-0.034938,-4.310660e-04,0.455133,0.0,0.025560,0.000018,0.034940,0.315119,0.112665
498,-0.09606,0.012390,-0.010591,-2.416680e-05,0.658054,0.0,0.006848,0.000016,0.010591,0.078314,0.096856


In [26]:
df6

,Cx,Cy,u,v,strain,boundary,T,cellVolume,U,wallinfo,rD
301,0.04804,0.006144,-0.006122,-8.807180e-06,0.648501,0.0,0.004006,0.760521,0.006122,0.046631,0.048431
2298,-0.09597,0.073409,-0.044150,-5.957790e-04,0.442379,0.0,0.031424,0.893521,0.044154,0.387876,0.120827
698,-0.09605,0.018737,-0.014698,-4.892190e-05,0.635802,0.0,0.009620,0.785448,0.014698,0.110516,0.097861
3100,-0.00005,0.103530,-0.053781,-1.189460e-03,0.319533,0.0,0.041797,0.952879,0.053794,0.540684,0.103530
2900,-0.00005,0.095815,-0.051236,-1.028280e-03,0.346163,0.0,0.039117,0.937665,0.051247,0.501546,0.095815
1898,-0.09599,0.059067,-0.037456,-3.959980e-04,0.493188,0.0,0.026090,0.865197,0.037458,0.315119,0.112708
2901,0.04790,0.095815,-0.050260,-1.048050e-03,0.337362,0.0,0.039014,0.937736,0.050271,0.501547,0.107121
2698,-0.09595,0.088225,-0.050293,-8.426110e-04,0.390095,0.0,0.036697,0.922761,0.050300,0.463039,0.130346
1902,0.09594,0.059067,-0.034938,-4.310660e-04,0.455133,0.0,0.025560,0.865120,0.034940,0.315119,0.112665
498,-0.09606,0.012390,-0.010591,-2.416680e-05,0.658054,0.0,0.006848,0.772887,0.010591,0.078314,0.096856


In [28]:
df7

,Cx,Cy,u,v,strain,boundary,cellVolume,U,wallinfo,rD,rfianl
301,0.04804,0.006144,-0.006122,-8.807180e-06,0.648501,0.0,0.760521,0.006122,0.046631,0.048431,0.001070
2298,-0.09597,0.073409,-0.044150,-5.957790e-04,0.442379,0.0,0.893521,0.044154,0.387876,0.120827,0.000445
698,-0.09605,0.018737,-0.014698,-4.892190e-05,0.635802,0.0,0.785448,0.014698,0.110516,0.097861,0.000047
3100,-0.00005,0.103530,-0.053781,-1.189460e-03,0.319533,0.0,0.952879,0.053794,0.540684,0.103530,0.002539
2900,-0.00005,0.095815,-0.051236,-1.028280e-03,0.346163,0.0,0.937665,0.051247,0.501546,0.095815,0.002590
1898,-0.09599,0.059067,-0.037456,-3.959980e-04,0.493188,0.0,0.865197,0.037458,0.315119,0.112708,0.000311
2901,0.04790,0.095815,-0.050260,-1.048050e-03,0.337362,0.0,0.937736,0.050271,0.501547,0.107121,0.003253
2698,-0.09595,0.088225,-0.050293,-8.426110e-04,0.390095,0.0,0.922761,0.050300,0.463039,0.130346,0.000583
1902,0.09594,0.059067,-0.034938,-4.310660e-04,0.455133,0.0,0.865120,0.034940,0.315119,0.112665,0.002717
498,-0.09606,0.012390,-0.010591,-2.416680e-05,0.658054,0.0,0.772887,0.010591,0.078314,0.096856,0.000029
